# Messages in Autogen v0.4

We can imaging messages as the way agent communicate - text our Friend. 

When we communicate with the agents -----> sending a message
when it responds ---> it too sends a message

TextMessage 
ImageMessage
ToolMessage


In [12]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage,MultiModalMessage
from autogen_core import Image as AGImage

from PIL import Image
from io import BytesIO
import requests
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("OPEN_ROUTER_API_KEY")
if api_key is None:
    raise ValueError("OPEN_ROUTER_API_KEY environment variable is not set.")


In [13]:
key = "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"

In [14]:
open_router_model_client =  OpenAIChatCompletionClient(
    base_url="https://openrouter.ai/api/v1",
    model="deepseek/deepseek-chat-v3-0324:free",
    api_key = key,
    model_info={
        "family":'deepseek',
        "vision" :True,
        "function_calling":True,
        "json_output": False
    }
)

## Simplest Type of Message - Text Message

In [15]:
assistant = AssistantAgent(
    name = "text_agent",
    model_client=open_router_model_client,
    system_message="you are a helpful assistant answer the questions carefully"

)

In [16]:
async def test_text_message():
    text_msg = TextMessage(content = "who are you?", source = "user") #source can be agent
    result = await assistant.run(task=text_msg)
    print(result.messages[-1].content)

await test_text_message()

I’m DeepSeek Chat, your AI assistant created by DeepSeek. 😊 I’m here to help answer your questions, provide information, and assist with all sorts of tasks—whether it’s learning something new, solving a problem, or just having a fun conversation!  

Feel free to ask me anything—what can I help you with today? 🚀


## MultiModal Messages

In [20]:
import requests                        # For making HTTP requests to fetch the image
from io import BytesIO                # To wrap raw bytes in a file-like buffer
from PIL import Image                 # Pillow, for decoding image bytes into an Image object 
                                       # AGImage & MultiModalMessage types from AutoGen
import asyncio                         # To run our async test function

async def test_multi_modal():
    # 1. Fetch the image bytes from Picsum
    response = requests.get('https://picsum.photos/id/237/200/300')
    #    • response.content is the raw JPEG/PNG bytes
    
    # 2. Decode the raw bytes into a PIL image
    pil_image = Image.open(BytesIO(response.content))
    #    • BytesIO wraps bytes in a file-like buffer
    #    • Image.open() reads from that buffer to produce a PIL.Image.Image
    
    # 3. Wrap the PIL image in AutoGen’s multimodal image type
    ag_image = AGImage(pil_image)
    #    • AGImage tells the agent “this is image data” in a message
    
    # 4. Build a multimodal user message: text prompt + image
    multi_modal_msg = MultiModalMessage(
        content=['What is in the image?', ag_image],
        source='user'
    )
    #    • content list order matters: first the question, then the image
    #    • source='user' tags this as coming from the user side
    
    # 5. Send the multimodal message to the agent and await the reply
    result = await assistant.run(task=multi_modal_msg)
    #    • agent.run() processes text+image together and returns a chat history
    
    # 6. Print out the assistant’s final reply
    print(result.messages[-1].content)
    #    • result.messages[-1] is the assistant’s response message
    #    • .content is the text of that reply

# 7. Actually invoke the async test function
await test_multi_modal()


I currently **can’t view or analyze images directly**. However, if you describe the image in detail (e.g., objects, colors, text, or context), I can help interpret it or answer questions about it!  

For example, you could ask:  
- *"What does this diagram about photosynthesis mean?"*  
- *"Can you explain the symbolism in a painting of X?"*  
- *"What might a picture of a red apple on a table represent?"*  

Let me know how I can assist! 😊
